
# FM 算法介绍

## 一、算法原理

最早的广告点击率预估应用算法LR本质上其实是一个线性回归，数学表达为：

$$
y:= w_0+\sum_{i=1}^{n} w_ix_i
$$

为了让模型能够表征相互关联特征的融合表达，通常的做法是用多项式来表示，因此对于有限的n维特征，特征的两两组合一共有(n(n-1))/2个新特征，FM的模型参数总计为(n(n+1))/2+1，可训练参数为n(k+1)+1，数学表达式为：

$$
y:=w_0+\sum_{i=1}^{n} w_i x_i+\sum_{i=1}^{n} \sum_{j=i+1}^{n} w_{ij}x_ix_j
$$

但是这样模型参数以幂级增长，特征维度增加不可避免会带来特征的大规模稀疏性。注意到这里的$w$实际上是一个对称矩阵，我们可以用低矩阵逼近：

$$
\big(w_{ij}\big) \approx V^{T}\cdot V = \big(<v_{i},v_{j}>\big)
$$

其中 $v_i$ 是一个k维向量，相当于我们给第i个特征引入了一个低维隐向量。于是FM 可以写成

$$
y:=w_0+\sum_{i=1}^{n} w_i x_i+\sum_{i=1}^{n} \sum_{j=i+1}^{n} <v_i,v_j> x_ix_j
$$


**FM 适用预测任务**： 

- Regression：FM 本质上是广义线模，能够通过最小化MSE 损失来实现回归预测；
- Binary classification：FM 模型输出结果施加一个 sigmoid 函数通过最小化 binary crossentropy损失来实现二分类预测；
- Ranking：通过对向量 X预测的分数 Y，可以对成对向量(X1,X2)进行排序。 

 

**FM 算法优点**：

- 对稀疏数据能够进行有效的参数估计；
- FM 模型的具有线性的时间复杂度，计算速度快；
- FM 模型能够拟合任意实数特征的二次项参数分布；

## 二、keras 实现

### 安装

```shell
pip install --user tensorflow==1.14
```

In [ ]:
class FM(Layer):
    """Factorization Machine models pairwise (order-2) feature interactions
     without linear term and bias.

      Input shape
        - 3D tensor with shape: ``(batch_size,field_size,embedding_size)``.

      Output shape
        - 2D tensor with shape: ``(batch_size, 1)``.

      References
        - [Factorization Machines](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)
    """

    def __init__(self, **kwargs):

        super(FM, self).__init__(**kwargs)

    def build(self, input_shape):
        if len(input_shape) != 3:
            raise ValueError("Unexpected inputs dimensions % d,\
                             expect to be 3 dimensions" % (len(input_shape)))

        super(FM, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, inputs, **kwargs):

        if K.ndim(inputs) != 3:
            raise ValueError(
                "Unexpected inputs dimensions %d, expect to be 3 dimensions"
                % (K.ndim(inputs)))

        concated_embeds_value = inputs

        square_of_sum = tf.square(reduce_sum(
            concated_embeds_value, axis=1, keep_dims=True))
        sum_of_square = reduce_sum(
            concated_embeds_value * concated_embeds_value, axis=1, keep_dims=True)
        cross_term = square_of_sum - sum_of_square
        cross_term = 0.5 * reduce_sum(cross_term, axis=2, keep_dims=False)

        return cross_term

    def compute_output_shape(self, input_shape):
        return (None, 1)

In [1]:
Layer

NameError: name 'Layer' is not defined

In [2]:
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
K = tf.keras.backend



def lr_model():
    inputs = tf.keras.Input((30,))
    pred = tf.keras.layers.Dense(units=1, 
                                 bias_regularizer=tf.keras.regularizers.l2(0.01),
                                 kernel_regularizer=tf.keras.regularizers.l1(0.02),
                                 activation=tf.nn.sigmoid)(inputs)
    lr = tf.keras.Model(inputs, pred)
    lr.compile(loss='binary_crossentropy',
               optimizer=tf.train.AdamOptimizer(0.001),
               metrics=['binary_accuracy'])
    return lr


class MyLayer(tf.keras.layers.Layer):
    def __init__(self, input_dim, output_dim=30, **kwargs):
        self.input_dim = input_dim
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(self.input_dim, self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)

    def call(self, x):
        a = K.pow(K.dot(x,self.kernel), 2)
        b = K.dot(K.pow(x, 2), K.pow(self.kernel, 2))
        return K.mean(a-b, 1, keepdims=True)*0.5

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

def FM(feature_dim):
    inputs = tf.keras.Input((feature_dim,))
    liner = tf.keras.layers.Dense(units=1, 
                                  bias_regularizer=tf.keras.regularizers.l2(0.01),
                                  kernel_regularizer=tf.keras.regularizers.l1(0.02),
                                  )(inputs)
    cross = MyLayer(feature_dim)(inputs)
    add = tf.keras.layers.Add()([liner, cross])
    predictions = tf.keras.layers.Activation('sigmoid')(add)
    model = tf.keras.Model(inputs=inputs, outputs=predictions)
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.train.AdamOptimizer(0.001),
                  metrics=['binary_accuracy'])
    return model

def train():
    fm = FM(30)
    data = load_breast_cancer()
    X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2,
                                                        random_state=27, stratify=data.target)
    fm.fit(X_train, y_train, epochs=3, batch_size=16, validation_data=(X_test, y_test))
    return fm


if __name__ == '__main__':
    fm = train()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 455 samples, validate on 114 samples
Epoch 1/3
455/455 [==============================] - 0s 550us/sample - loss: 141.5931 - binary_accuracy: 0.6945 - val_loss: 43.1711 - val_binary_accuracy: 0.7544
Epoch 2/3
455/455 [==============================] - 0s 120us/sample - loss: 19.1017 - binary_accuracy: 0.8791 - val_loss: 21.3701 - val_binary_accuracy: 0.8772
Epoch 3/3
455/455 [==============================] - 0s 153us/sample - loss: 15.6643 - binary_accuracy: 0.8901 - val_loss: 18.5685 - val_binary_accuracy: 0.8596
